In [196]:
# from kan2 import * 不可解释
#暂时用这个 可解释
from kan import KAN
from sklearn.model_selection import KFold, cross_val_score, train_test_split
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons
import torch
from sklearn.utils import shuffle
import random
import numpy as np
torch.set_default_dtype(torch.float32)

In [214]:
protein_feature_train = []

# 打开并读取文件
with open('/tmp/pycharm_project_763/feature/feature_train/phychem_train.txt', 'r') as file:
    # 逐行读取文件内容
    for line in file:
        if line.startswith('>'):  # 跳过以'>'开头的行（序列标识行）
            continue
        # 分割行中的每个特征并转换为浮点数
        features = [float(x) for x in line.strip().split('\t')]
        # 将特征列表添加到主列表中
        protein_feature_train.append(features)
# protein_feature_train[:5]   

In [215]:
protein_feature_test = []

# 打开并读取文件
with open('/tmp/pycharm_project_763/feature/feature_test/phychem_test.txt', 'r') as file:
    # 逐行读取文件内容
    for line in file:
        if line.startswith('>'):  # 跳过以'>'开头的行（序列标识行）
            continue
        # 分割行中的每个特征并转换为浮点数
        features = [float(x) for x in line.strip().split('\t')]
        # 将特征列表添加到主列表中
        protein_feature_test.append(features)

In [216]:
import pandas as pd
df = pd.read_csv('/tmp/pycharm_project_763/data/trainCPP.csv')

# 提取序列和标签
sequences = df['sequence'].tolist()
y = df['label'].tolist()

df2= pd.read_csv('/tmp/pycharm_project_763/data/testCPP.csv')

# 提取序列和标签
sequences_test = df2['sequence'].tolist()
y_test = df2['label'].tolist()

In [217]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(protein_feature_train, y, random_state=42)
X_test, y_test = shuffle(protein_feature_test, y_test, random_state=42)
# X_train[:5],y_train[:5]

In [218]:
X_train=np.array(X_train)
X_test=np.array(X_test)
y_train=np.array(y_train)
y_test=np.array(y_test)

In [219]:

# def one_hot_encode(arr):
#     # 创建一个形状为(len(arr), 2)的全零数组
#     one_hot = np.zeros((arr.size, 2))
#     # 对于数组中的每个1，我们设置第二列为1；对于0，则设置第一列为1
#     one_hot[np.arange(arr.size), arr] = 1
#     return one_hot
# 
# # 应用独热编码
# y_test = one_hot_encode(y_test)
# y_train = one_hot_encode(y_train)



X_train = torch.tensor(X_train)
y_train = torch.tensor(y_train).view(-1, 1)
X_test = torch.tensor(X_test)
y_test = torch.tensor(y_test).view(-1, 1)
# X_train = torch.tensor(X_train ,dtype=torch.float32)
# y_train = torch.tensor(y_train,dtype=torch.float32)
# X_test = torch.tensor(X_test,dtype=torch.float32)
# y_test = torch.tensor(y_test,dtype=torch.float32)

In [220]:
# X_train = torch.from_numpy(X_train).to(torch.int32)
# y_train = torch.from_numpy(y_train).to(torch.int32)
# X_test = torch.from_numpy(X_test).to(torch.int32)
# y_test = torch.from_numpy(y_test).to(torch.int32)


In [221]:
# def remove_all_zero_rows(X, y):
#     # 找到不是全零的行的索引
#     non_zero_indices = ~(X == 0).all(dim=1)
#     # 过滤掉全零的行
#     X_filtered = X[non_zero_indices]
#     y_filtered = y[non_zero_indices]
#     return X_filtered, y_filtered
# 
# # 删除全零行
# X_train, y_train = remove_all_zero_rows(X_train, y_train)
# X_test, y_test = remove_all_zero_rows(X_test, y_test)

In [222]:
# y_train = y_train.reshape((y_train.shape[0], 1))
# y_test = y_test.reshape((y_test.shape[0], 1))

dataset={}
dataset = {
        'train_input': X_train,
        'test_input': X_test,
        'train_label': y_train,
        'test_label': y_test
    }

In [223]:
dataset['train_label'].shape

torch.Size([1164, 2])

In [224]:
dataset['test_label'].shape

torch.Size([299, 2])

In [225]:
#如果明确是二分类问题且模型不需要扩展，使用一个神经元的设计更为简单和高效。
#如果可能会扩展为多分类问题或在某些实现中softmax更稳定，则可以使用两个神经元的设计。
model = KAN(width=[24,5,1], grid=3, k=2, seed=42)  #对应导入kan 可以进行剪zhi和一系列操作
# grid=5,k=3
# model = KAN(width=[24,5,2], grid=3, k=2, seed=42)  对应导入kan2 似乎准确率更高 过拟合更好一点
# model = model.to(torch.float32)

checkpoint directory created: ./model
saving model version 0.0


In [227]:
from sklearn.metrics import roc_auc_score, confusion_matrix, matthews_corrcoef
def train_acc():
    # pred=model(X_train)
    # with torch.no_grad():
    #     pred_labels = (pred > 0.5).float() 
    #     auc = roc_auc_score(y_train.cpu(), pred.cpu())
    # 
    # # 混淆矩阵计算
    #     tn, fp, fn, tp = confusion_matrix(y_train.cpu(), pred_labels.cpu()).ravel()
    # 
    #     # Specificity (SP)
    #     sp = tn / (tn + fp) if (tn + fp) > 0 else 0
    # 
    #     # Sensitivity (SN)
    #     sn = tp / (tp + fn) if (tp + fn) > 0 else 0
    # 
    #     # Matthews Correlation Coefficient (MCC)
    #     mcc = matthews_corrcoef(y_train.cpu(), pred_labels.cpu())
    # print(f"AUC: {auc:.4f}, SP: {sp:.4f}, MCC: {mcc:.4f}, SN: {sn:.4f}")
    return torch.mean((torch.round(model(X_train)[:, 0]) == y_train[:, 0]).float())

def test_acc():
    return torch.mean((torch.round(model(X_test)[:, 0]) == y_test[:, 0]).float())

# results = model.train(dataset, opt="LBFGS", steps=10, metrics=(train_acc, test_acc)) ,lamb=0.001 lamb_entropy=4.,lamb=0.1,lamb_l1=2.5,
# lamb=0.005 train/fit
results = model.fit(dataset, opt="LBFGS", steps=5, metrics=(train_acc, test_acc));
print(results['train_acc'][-1], results['test_acc'][-1])

| train_loss: 3.77e-01 | test_loss: 4.36e-01 | reg: 0.00e+00 | : 100%|█| 5/5 [00:02<00:00,  2.19it/s

saving model version 0.1
0.7860824465751648 0.692307710647583


In [230]:
model.plot()

cannot plot since data are not saved. Set save_act=True first.


IndexError: list index out of range

In [231]:
model=model.prune()
model.plot()
#kan2报错 kan没问题

IndexError: list index out of range

In [ ]:
results =model.fit(dataset, opt="LBFGS", steps=5, metrics=(train_acc, test_acc));

In [ ]:
print(results['train_acc'][-1], results['test_acc'][-1])

In [ ]:
model = model.refine(10)

In [ ]:
model.plot()

In [ ]:
results =model.fit(dataset, opt="LBFGS", steps=5, metrics=(train_acc, test_acc));

In [ ]:
print(results['train_acc'][-1], results['test_acc'][-1])

In [ ]:
model.plot()

In [109]:
mode = "auto" # "manual"
"""
假设在一个神经网络模型中，坐标 (0,36,4) 可能表示：

层索引 (0): 第一层或第一个处理阶段。
单元索引 (36): 第一层中的第 37 个单元（因为索引从 0 开始）。
通道或特征索引 (4): 第 5 个通道或特征
"""

if mode == "manual":
    # manual mode
    model.fix_symbolic(0,0,0,'sin');
    model.fix_symbolic(0,1,0,'x^2');
    model.fix_symbolic(1,0,0,'exp');
elif mode == "auto":
    # automatic mode
    lib = ['x','x^2','x^3','x^4','exp','log','sqrt','tanh','sin','abs']
    model.auto_symbolic(lib=lib)

AttributeError: 'KAN' object has no attribute 'acts'

In [ ]:
from kan.utils import ex_round

ex_round(model.symbolic_formula()[0][0],4)